In [1]:
import numpy as np
import librosa as rs

import matplotlib.pyplot as plt   
import IPython.display as ipd

In [2]:
def tailor_dB_FS(y, target_dB_FS=-25, eps=1e-6):
    rms = np.sqrt(np.mean(y ** 2))
    scalar = 10 ** (target_dB_FS / 20) / (rms + eps)
    y *= scalar
    return y, rms, scalar

In [8]:
def mix(clean,noise, SNR, dB):
    target_dB_FS = -25
    eps = 1e-13
    
    clean = clean/(np.max(np.abs(clean)) + eps)
    noise = noise/(np.max(np.abs(noise)) + eps)
    
    # scale
    clean, _, _ = tailor_dB_FS(clean, target_dB_FS)
    noise, _, _ = tailor_dB_FS(noise, target_dB_FS)
    
    clean_rms = (clean ** 2).mean() ** 0.5
    noise_rms = (noise ** 2).mean() ** 0.5
    
    # SNR
    snr_scalar = clean_rms / (10 ** (SNR / 20)) / (noise_rms + eps)
    noise *= snr_scalar
    # Mix
    noisy = clean + noise
    
    # rescale noisy RMS
    noisy, _, noisy_scalar = tailor_dB_FS(noisy, dB)
    clean *= noisy_scalar
    return noisy,clean,noise

In [11]:
c = rs.load("data/female_1.wav",sr=16000)[0]
n = rs.load("data/cafe.wav",sr=16000)[0]

print(c.shape)
print(n.shape)

n = n[:len(c)]
print(n.shape)

noisy,clean,noise = mix(c,n,10,-25)
ipd.display(ipd.Markdown('## input'))
ipd.display(ipd.Audio(noisy, rate=16000))

(29390,)
(164146,)
(29390,)


## input